# Data Validation and Cleaning

## Load Data

In [1]:
# -*- coding: utf-8 -*-
import pandas as pd
import numpy as np
from pathlib import Path
import re
from collections import Counter

# ========= CONFIG =========
# กำหนด root path ของโฟลเดอร์โปรเจกต์
ROOT_DIR = Path("/Users/thanwaratkeratipasuwat/Desktop/dsi314")

# พาธข้อมูลต้นทาง (Raw Data)
RAW_DIR  = ROOT_DIR / "Raw data"

BKF_PATH  = RAW_DIR / "bkf_eflora_species_dedup.csv"   # BKF หลัง dedup
#OUT7_PATH = RAW_DIR / "output (7).csv"                 # จาก WFO map tools
#CAND_PATH = RAW_DIR / "candidates (3).csv"             # candidates เพิ่มเติม

# พาธผลลัพธ์ (Processed Data)
OUT_DIR   = ROOT_DIR / "Processed data"
#OUT_DIR.mkdir(parents=True, exist_ok=True)  # สร้างโฟลเดอร์ถ้ายังไม่มี

# กำหนดชื่อไฟล์ผลลัพธ์
# OUT_CLEAN_PATH   = OUT_DIR / "bkf_full_after_scrape.csv"        # ข้อมูลที่เติมชื่อแล้ว
# OUT_VALID_PATH   = OUT_DIR / "bkf_full_after_clean.csv"         # หลัง validation + cleaning
# OUT_REVIEW_PATH  = OUT_DIR / "bkf_multicand_review_queue.csv"   # แถวที่ต้องตรวจ manual

# แสดงพาธทั้งหมดเพื่อตรวจสอบ
print("📁 Paths Configuration:")
print(f"BKF_PATH         : {BKF_PATH}")
#print(f"OUT7_PATH        : {OUT7_PATH}")
#print(f"CAND_PATH        : {CAND_PATH}")
print(f"OUT_DIR          : {OUT_DIR}")
# print(f"OUT_CLEAN_PATH   : {OUT_CLEAN_PATH}")
# print(f"OUT_VALID_PATH   : {OUT_VALID_PATH}")
# print(f"OUT_REVIEW_PATH  : {OUT_REVIEW_PATH}")

📁 Paths Configuration:
BKF_PATH         : /Users/thanwaratkeratipasuwat/Desktop/dsi314/Raw data/bkf_eflora_species_dedup.csv
OUT_DIR          : /Users/thanwaratkeratipasuwat/Desktop/dsi314/Processed data


In [2]:
# ========= HELPERS =========
def norm(x):
    if pd.isna(x): return ""
    return str(x).replace("\u00A0", " ").strip().lower()

def find_col(cols, candidates):
    """หาคอลัมน์แบบทนทานต่อเคสตัวพิมพ์"""
    cols_list = list(cols)
    # หาแบบตรงชื่อก่อน
    for c in candidates:
        if c in cols_list:
            return c
    # หาแบบ lower-case map
    lower_map = {c.lower(): c for c in cols_list}
    for c in candidates:
        if c.lower() in lower_map:
            return lower_map[c.lower()]
    return None

def nonempty_series_like(df, col):
    """คืน Series ของคอลัมน์ (ถ้าไม่มีให้คืนซีรีส์ว่าง) และแปลง nan/None เป็น '' """
    if col is None or col not in df.columns:
        return pd.Series("", index=df.index)
    return (df[col].astype(str)
                 .str.strip()
                 .replace({"nan": "", "None": "", "NaT": ""}))

def wfo_clean(x):
    """ทำความสะอาด WFO -> รูปแบบ wfo-xxxx (ตัวพิมพ์เล็ก, ตัดอักขระแปลก)"""
    if pd.isna(x): return ""
    s = str(x).strip()
    if s == "" or s.lower() in {"none","nan"}:
        return ""
    s2 = s.lower()
    if not s2.startswith("wfo-"):
        s2 = "wfo-" + s2
    s2 = re.sub(r"[^a-z0-9\-]", "", s2)
    return s2

def prefer(left, right):
    """เลือก left ถ้าไม่ว่าง มิฉะนั้นใช้ right"""
    ls = left.astype(str).str.strip()
    rs = right.astype(str).str.strip()
    return np.where(ls != "", ls, rs)

def resolve_eff(df, colname, suffix):
    """คืนชื่อคอลัมน์ที่มีอยู่จริงหลัง merge (เช็คว่ามีการเติม suffix หรือไม่)"""
    if not colname:
        return None
    if f"{colname}{suffix}" in df.columns:
        return f"{colname}{suffix}"
    return colname if colname in df.columns else None

In [3]:
# ========= LOAD =========
df_bkf = pd.read_csv(BKF_PATH)
# df_o7  = pd.read_csv(OUT7_PATH)
# df_c   = pd.read_csv(CAND_PATH)

display(df_bkf.head(3))#, df_o7.head(3), df_c.head(3))


,volume,family_name,genus_name,genus_label,genus_index_parsed,species_scientific_name,accepted_name,thailand,distribution,ecology,family_url,genus_url,species_url,scraped_at,specific_name,wfo_id
0,2,Haloragaceae,1 Myriophyllum,Myriophyllum,1.0,Myriophyllum siamense (Craib) Tardieu,This is currently accepted.,"PENINSULAR: Nakhon Si Thammarat, Songkhla (type).","Known from 3 localities: the type-locality, an...","In small mats on damp sandy ground, edge of ma...",https://botany.dnp.go.th/eflora/florafamily.ht...,https://botany.dnp.go.th/eflora/floragenus.htm...,https://botany.dnp.go.th/eflora/floraspecies.h...,2025-10-19T18:17:33.566587+00:00,Myriophyllum siamense (Craib) Tardieu,NaN
1,2,Haloragaceae,1 Myriophyllum,Myriophyllum,1.0,Myriophyllum tetrandrum Roxb.,This is currently accepted.,SOUTH-WESTERN: Prachuap Khiri Khan (Bang Sapha...,"E India (type), Indochina, Malay Peninsula.","In rather shallow, open water of ditches, cana...",https://botany.dnp.go.th/eflora/florafamily.ht...,https://botany.dnp.go.th/eflora/floragenus.htm...,https://botany.dnp.go.th/eflora/floraspecies.h...,2025-10-19T18:17:36.633398+00:00,Myriophyllum tetrandrum Roxb.,wfo-0001261381
2,2,Haloragaceae,2 Haloragis,Haloragis,2.0,Haloragis micrantha (Thunb.) R.Br. ex Sieb. & ...,Gonocarpus micranthus,NORTH-EASTERN: Loei (Phu Kradueng).,"India, S & E China, N Vietnam, Hainan, Formosa...","In marshy mountain turf, moist places along mo...",https://botany.dnp.go.th/eflora/florafamily.ht...,https://botany.dnp.go.th/eflora/floragenus.htm...,https://botany.dnp.go.th/eflora/floraspecies.h...,2025-10-19T18:17:50.049475+00:00,Gonocarpus micranthus,wfo-0000715064


In [4]:
# ========= IDENTIFY KEY COLUMNS =========
bkf_spec = find_col(df_bkf.columns, ["specific_name"])
bkf_wfo  = find_col(df_bkf.columns, ["wfo_id"])
bkf_url  = find_col(df_bkf.columns, ["species_url"])
bkf_full = find_col(df_bkf.columns, ["wfo_full_name","wfo_name","full_name","accepted_name"])

if bkf_spec is None:
    raise KeyError("ไม่พบคอลัมน์ specific_name ใน BKF")

if bkf_wfo is None:
    # ถ้า BKF ไม่มีคอลัมน์ WFO ให้สร้างว่างไว้
    df_bkf["wfo_id"] = np.nan
    bkf_wfo = "wfo_id"

# o7_spec  = find_col(df_o7.columns, ["specific_name"])
# o7_wfo   = find_col(df_o7.columns, ["wfo_id","wfo"])
# o7_full  = find_col(df_o7.columns, ["wfo_full_name"])

# c_spec   = find_col(df_c.columns, ["specific_name"])
# c_wfo    = find_col(df_c.columns, ["wfo_id"])
# c_full   = find_col(df_c.columns, ["wfo_full_name"])

In [5]:
# ========= NORMALIZE KEY & WFO =========
df_bkf["specific_name_clean"] = df_bkf[bkf_spec].apply(norm)
if bkf_url: df_bkf["species_url_clean"] = df_bkf[bkf_url].apply(norm)

# if o7_spec: df_o7["specific_name_clean"] = df_o7[o7_spec].apply(norm)
# if c_spec:  df_c["specific_name_clean"]  = df_c[c_spec].apply(norm)

# ทำความสะอาด WFO ในทุกตารางที่มี
df_bkf[bkf_wfo] = nonempty_series_like(df_bkf, bkf_wfo).apply(wfo_clean)
# if o7_wfo:
#     df_o7[o7_wfo] = nonempty_series_like(df_o7, o7_wfo).apply(wfo_clean)
# if c_wfo:
#     df_c[c_wfo]   = nonempty_series_like(df_c, c_wfo).apply(wfo_clean)

### STEP (0) : **Missing `wfo_id` Check**

In [6]:
# ========= STEP 0: BASELINE =========
missing_before = df_bkf[bkf_wfo].eq("") | df_bkf[bkf_wfo].isna()
print(f"[BASELINE] Missing WFO (BKF): {int(missing_before.sum())} / {len(df_bkf)}")

[BASELINE] Missing WFO (BKF): 898 / 7469


>   นับจำนวนแถวใน BKF ที่ยัง ไม่มี wfo_id ("" หรือ NaN).

In [7]:
# แถวที่ยังไม่มี wfo_id (missing)
df_bkf_missing = df_bkf[missing_before].copy()
print(f"Missing WFO (BKF): {len(df_bkf_missing)} แถว")

# แถวที่มี wfo_id แล้ว (not missing)
df_bkf_present = df_bkf[~missing_before].copy()
print(f"มี WFO แล้ว: {len(df_bkf_present)} แถว")

# ตัวอย่างตรวจสอบ
print("\nตัวอย่าง missing:")
display(df_bkf_missing.head(3))

print("\nตัวอย่างที่มี WFO แล้ว:")
display(df_bkf_present.head(3))

Missing WFO (BKF): 898 แถว
มี WFO แล้ว: 6571 แถว

ตัวอย่าง missing:


,volume,family_name,genus_name,genus_label,genus_index_parsed,species_scientific_name,accepted_name,thailand,distribution,ecology,family_url,genus_url,species_url,scraped_at,specific_name,wfo_id,specific_name_clean,species_url_clean
0,2,Haloragaceae,1 Myriophyllum,Myriophyllum,1.0,Myriophyllum siamense (Craib) Tardieu,This is currently accepted.,"PENINSULAR: Nakhon Si Thammarat, Songkhla (type).","Known from 3 localities: the type-locality, an...","In small mats on damp sandy ground, edge of ma...",https://botany.dnp.go.th/eflora/florafamily.ht...,https://botany.dnp.go.th/eflora/floragenus.htm...,https://botany.dnp.go.th/eflora/floraspecies.h...,2025-10-19T18:17:33.566587+00:00,Myriophyllum siamense (Craib) Tardieu,,myriophyllum siamense (craib) tardieu,https://botany.dnp.go.th/eflora/floraspecies.h...
4,2,Rhizophoraceae,1 Rhizophora,Rhizophora,1.0,Rhizophora mucronata Poir.,This is currently accepted.,SOUTH-EASTERN: Chanthaburi; SOUTH-WESTERN: Pra...,"In the Old World tropics, occurring from the c...",Mangrove forests.,https://botany.dnp.go.th/eflora/florafamily.ht...,https://botany.dnp.go.th/eflora/floragenus.htm...,https://botany.dnp.go.th/eflora/floraspecies.h...,2025-10-19T18:18:19.615290+00:00,Rhizophora mucronata Poir.,,rhizophora mucronata poir.,https://botany.dnp.go.th/eflora/floraspecies.h...
5,2,Rhizophoraceae,2 Bruguiera,Bruguiera,2.0,Bruguiera gymnorhiza (L.) Savigny,This is currently accepted.,"SOUTH-EASTERN: Chanthaburi, Trat; PENINSULAR: ...","In the Old World tropics, from S & E Africa th...",Mangrove forests.,https://botany.dnp.go.th/eflora/florafamily.ht...,https://botany.dnp.go.th/eflora/floragenus.htm...,https://botany.dnp.go.th/eflora/floraspecies.h...,2025-10-19T18:18:33.101996+00:00,Bruguiera gymnorhiza (L.) Savigny,,bruguiera gymnorhiza (l.) savigny,https://botany.dnp.go.th/eflora/floraspecies.h...



ตัวอย่างที่มี WFO แล้ว:


,volume,family_name,genus_name,genus_label,genus_index_parsed,species_scientific_name,accepted_name,thailand,distribution,ecology,family_url,genus_url,species_url,scraped_at,specific_name,wfo_id,specific_name_clean,species_url_clean
1,2,Haloragaceae,1 Myriophyllum,Myriophyllum,1.0,Myriophyllum tetrandrum Roxb.,This is currently accepted.,SOUTH-WESTERN: Prachuap Khiri Khan (Bang Sapha...,"E India (type), Indochina, Malay Peninsula.","In rather shallow, open water of ditches, cana...",https://botany.dnp.go.th/eflora/florafamily.ht...,https://botany.dnp.go.th/eflora/floragenus.htm...,https://botany.dnp.go.th/eflora/floraspecies.h...,2025-10-19T18:17:36.633398+00:00,Myriophyllum tetrandrum Roxb.,wfo-0001261381,myriophyllum tetrandrum roxb.,https://botany.dnp.go.th/eflora/floraspecies.h...
2,2,Haloragaceae,2 Haloragis,Haloragis,2.0,Haloragis micrantha (Thunb.) R.Br. ex Sieb. & ...,Gonocarpus micranthus,NORTH-EASTERN: Loei (Phu Kradueng).,"India, S & E China, N Vietnam, Hainan, Formosa...","In marshy mountain turf, moist places along mo...",https://botany.dnp.go.th/eflora/florafamily.ht...,https://botany.dnp.go.th/eflora/floragenus.htm...,https://botany.dnp.go.th/eflora/floraspecies.h...,2025-10-19T18:17:50.049475+00:00,Gonocarpus micranthus,wfo-0000715064,gonocarpus micranthus,https://botany.dnp.go.th/eflora/floraspecies.h...
3,2,Rhizophoraceae,1 Rhizophora,Rhizophora,1.0,Rhizophora apiculata Blume,This is currently accepted.,CENTRAL: Chon Buri (Si Racha); SOUTH-EASTERN: ...,In tropical SE Asia throughout Malesia (type) ...,Mangrove forests.,https://botany.dnp.go.th/eflora/florafamily.ht...,https://botany.dnp.go.th/eflora/floragenus.htm...,https://botany.dnp.go.th/eflora/floraspecies.h...,2025-10-19T18:18:15.874773+00:00,Rhizophora apiculata Blume,wfo-0001131596,rhizophora apiculata blume,https://botany.dnp.go.th/eflora/floraspecies.h...


In [11]:
df_bkf_missing.to_csv("../Processed data/bkf_missing_wfo.csv", index=False)
df_bkf_present.to_csv("../Processed data/bkf_present_wfo.csv", index=False)
print("✅ Saved -> bkf_missing_wfo.csv และ bkf_present_wfo.csv")

✅ Saved -> bkf_missing_wfo.csv และ bkf_present_wfo.csv


In [12]:
df_bkf_present['wfo_id'].nunique()

6571

In [40]:
# -*- coding: utf-8 -*-
# ใช้ตัวแปรเดิมจากโน้ตบุ๊ก: df_bkf_present, bkf_wfo, bkf_full
import os, time, random
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import TimeoutException, WebDriverException

# =========================================================
# CONFIG
# =========================================================
BASE_URL     = "https://www.worldfloraonline.org/taxon/"
MAX_ATTEMPTS = 4            # จำนวนครั้ง retry ต่อหนึ่งไอดี
WAIT_SEC     = 20           # เวลา wait DOM
SAVE_EVERY   = 20           # เซฟ progress/cache ทุก N ไอดี
SLEEP_MIN    = 4.0          # หน่วงสุ่มเพื่อสุภาพ (กันโดน rate limit)
SLEEP_MAX    = 9.0

# โฟลเดอร์สำหรับบันทึกผล (แก้ให้ตรงเครื่องคุณ)
PROCESSED_DIR = "/Users/thanwaratkeratipasuwat/Desktop/dsi314/Processed data"
#os.makedirs(PROCESSED_DIR, exist_ok=True)
CACHE_CSV    = os.path.join(PROCESSED_DIR, "bkf_present_wf_cache.csv")
PROGRESS_CSV = os.path.join(PROCESSED_DIR, "bkf_present_wf_progress.csv")
OUT_CSV      = os.path.join(PROCESSED_DIR, "bkf_present_wf.csv")

# =========================================================
# SANITY CHECKS + เตรียมคอลัมน์เป้าหมาย
# =========================================================
# bkf_wfo ต้องเป็นชื่อคอลัมน์ (string)
if not isinstance(bkf_wfo, str):
    raise TypeError(f"bkf_wfo ควรเป็นชื่อคอลัมน์แบบ str แต่ได้ {type(bkf_wfo)}")

# ชื่อคอลัมน์สำหรับเก็บชื่อเต็ม
col_full = bkf_full if bkf_full else "wfo_full_name"
if col_full not in df_bkf_present.columns:
    df_bkf_present[col_full] = np.nan

def nonempty(s):
    return s.astype(str).str.strip().ne("")

# =========================================================
# เลือก "ทุก" WFO ID ที่ไม่ซ้ำใน df_bkf_present
# (ไม่กรองตาม miss_name เพื่อให้ดึงครบ 6571)
# =========================================================
ids_all_unique = (df_bkf_present[bkf_wfo]
                  .dropna().astype(str).str.strip()
                  .drop_duplicates()
                  .tolist())
print(f"🔢 Unique WFO IDs in df_bkf_present: {len(ids_all_unique)}")

# =========================================================
# โหลด/สร้าง CACHE: เก็บผล scrape (ok/err) เพื่อลดการยิงซ้ำ
# =========================================================
if os.path.exists(CACHE_CSV):
    cache_df = pd.read_csv(CACHE_CSV)
    if bkf_wfo not in cache_df.columns:
        # พยายาม rename คอลัมน์ที่ใกล้เคียง
        for alt in ["wfo_id","wfo","wfoid","wfo_code"]:
            if alt in cache_df.columns:
                cache_df = cache_df.rename(columns={alt: bkf_wfo})
                break
else:
    cache_df = pd.DataFrame(columns=[bkf_wfo, "scraped_name", "scrape_status"])

done_ok = set(cache_df.loc[cache_df["scrape_status"]=="ok", bkf_wfo]) if not cache_df.empty else set()
ids_to_scrape = [i for i in ids_all_unique if i not in done_ok]
print(f"🧭 To do after cache check: {len(ids_to_scrape)}")

# =========================================================
# ตั้งค่า Selenium (headless + polite UA)
# =========================================================
chrome_opts = Options()
chrome_opts.add_argument("--headless=new")      # comment บรรทัดนี้หากอยากเห็นหน้าจอจริง
chrome_opts.add_argument("--no-sandbox")
chrome_opts.add_argument("--disable-dev-shm-usage")
chrome_opts.add_argument("--disable-gpu")
chrome_opts.add_argument("--disable-blink-features=AutomationControlled")
chrome_opts.add_argument("--window-size=1280,900")
chrome_opts.add_argument("--lang=en-US")
chrome_opts.add_argument("--disable-features=NetworkService")
chrome_opts.add_argument("--user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X) "
                         "AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36")

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_opts)
driver.set_page_load_timeout(WAIT_SEC + 5)

# =========================================================
# Helpers
# =========================================================
def extract_name_from_html(html: str):
    soup = BeautifulSoup(html, "lxml")
    sci = soup.select_one("em[itemprop='scientificName'].taxonName")
    if not sci:
        sci = soup.select_one("em[itemprop='scientificName']") or soup.select_one("em.taxonName")
    auth = soup.select_one("span[itemprop='scientificNameAuthorship']")
    if not sci:
        return None
    sci_name  = sci.get_text(strip=True)
    auth_name = auth.get_text(strip=True) if auth else ""
    return f"{sci_name} {auth_name}".strip()

def polite_pause(extra=0.0):
    # หน่วงเวลาแบบสุ่มทุกครั้ง
    time.sleep(random.uniform(SLEEP_MIN, SLEEP_MAX) + extra)

def scrape_one_wfo(wfo_id: str):
    url = BASE_URL + str(wfo_id).strip()
    attempt = 0
    last_err = "unknown_error"
    while attempt < MAX_ATTEMPTS:
        attempt += 1
        try:
            driver.get(url)
            WebDriverWait(driver, WAIT_SEC).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, "em[itemprop='scientificName'], em.taxonName"))
            )
            time.sleep(0.8)  # เผื่อหน้าโหลดช้า
            full_name = extract_name_from_html(driver.page_source)
            if full_name:
                return full_name, "ok"
            raise TimeoutException("no_scientificName_tag")
        except (TimeoutException, WebDriverException) as e:
            last_err = f"retry_{attempt}: {e}"
            # backoff เพิ่มเมื่อ error
            backoff = min(2.0 * attempt, 8.0) + random.uniform(0.7, 1.7)
            if any(k in str(e) for k in ["ERR_CONNEC", "ERR_TIMED_OUT", "ERR_FAILED"]):
                backoff += 2.0
            polite_pause(extra=backoff)
        except Exception as e:
            return None, f"error: {e}"
    return None, last_err

# =========================================================
# RUN SCRAPE
# =========================================================
progress_rows = []
try:
    total = len(ids_to_scrape)
    print(f"🚀 Start scraping {total} WFO pages (polite mode)")
    for i, wfo_id in enumerate(ids_to_scrape, start=1):
        wfo_id = str(wfo_id).strip()
        if not wfo_id or wfo_id.lower() == "nan":
            progress_rows.append({bkf_wfo: wfo_id, "scraped_name": "", "scrape_status": "empty_wfo_id"})
            continue

        full_name, status = scrape_one_wfo(wfo_id)
        progress_rows.append({bkf_wfo: wfo_id, "scraped_name": full_name or "", "scrape_status": status})

        # หน่วงแบบสุภาพทุกครั้ง
        polite_pause()

        # Save cache/progress เป็นช่วง ๆ
        if (i % SAVE_EVERY == 0) or (i == total):
            new_df = pd.DataFrame(progress_rows)
            cache_df = pd.concat([cache_df, new_df], ignore_index=True)
            # ให้สถานะ ok ชนะกรณีซ้ำ
            cache_df.sort_values(by=["scrape_status"], key=lambda s: s.ne("ok"), inplace=True)
            cache_df = cache_df.drop_duplicates(subset=[bkf_wfo], keep="first")
            cache_df.to_csv(CACHE_CSV, index=False)
            new_df.to_csv(PROGRESS_CSV, index=False)
            print(f"💾 Progress saved ({i}/{total}) -> {PROGRESS_CSV}")
            progress_rows = []
finally:
    driver.quit()

# =========================================================
# MERGE กลับเข้า df_bkf_present แล้วเติมชื่อให้ครบทุกแถว
# =========================================================
ok_map = cache_df.loc[cache_df["scrape_status"]=="ok", [bkf_wfo, "scraped_name"]].rename(
    columns={"scraped_name": "__wfo_full_name_scraped__"}
)

# merge แบบ left เพื่อได้คอลัมน์ชื่อ scraped ชั่วคราว
df_bkf_present = df_bkf_present.merge(ok_map, on=bkf_wfo, how="left")

# เติมชื่อเต็มเฉพาะที่ยังว่าง/ว่างเปล่า
fill_mask = df_bkf_present[col_full].isna() | (~nonempty(df_bkf_present[col_full]))
df_bkf_present.loc[fill_mask, col_full] = df_bkf_present.loc[fill_mask, "__wfo_full_name_scraped__"]

# ลบคอลัมน์ช่วย
df_bkf_present.drop(columns=["__wfo_full_name_scraped__"], inplace=True)

# =========================================================
# SAVE ออกไฟล์ในโฟลเดอร์ Processed data
# =========================================================
df_bkf_present.to_csv(OUT_CSV, index=False)
print(f"✅ DONE. Saved df_bkf_present (with '{col_full}') -> {OUT_CSV}")

# ออปชัน: รายงานจำนวนแถวที่เติมชื่อได้ใหม่จากรอบนี้
already_had = nonempty(df_bkf_present[col_full]).sum()
print(f"✨ Rows with non-empty '{col_full}': {already_had} / {len(df_bkf_present)}")

🔢 Unique WFO IDs in df_bkf_present: 6571
🧭 To do after cache check: 3561
🚀 Start scraping 3561 WFO pages (polite mode)


KeyboardInterrupt: 

In [46]:
# -*- coding: utf-8 -*-
# ============= WFO FULL NAME SCRAPER — FAST, POLITE, RESUMABLE =============
# ใช้ตัวแปรเดิมจากโน้ตบุ๊ก: df_bkf_present, bkf_wfo, bkf_full

import os, sys, time, random, uuid, shutil
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, WebDriverException
from webdriver_manager.chrome import ChromeDriverManager

# ======================================================================
# CONFIG (ปรับให้เร็วขึ้น แต่ยังสุภาพพอไม่โดนบล็อก)
# ======================================================================
BASE_URL     = "https://www.worldfloraonline.org/taxon/"
MAX_ATTEMPTS = 4            # retry ต่อไอดี
WAIT_SEC     = 18           # รอ DOM
SAVE_EVERY   = 10           # << เซฟทุก 10 แถว
SLEEP_MIN    = 2.0          # << พักสุภาพสั้นลง
SLEEP_MAX    = 5.0

# โฟลเดอร์ผลลัพธ์ (ปรับ path ให้ตรงเครื่องคุณแล้วก่อนหน้า)
PROCESSED_DIR = "/Users/thanwaratkeratipasuwat/Desktop/dsi314/Processed data"
CACHE_CSV     = os.path.join(PROCESSED_DIR, "bkf_present_wf_cache.csv")
PROGRESS_CSV  = os.path.join(PROCESSED_DIR, "bkf_present_wf_progress.csv")
OUT_CSV       = os.path.join(PROCESSED_DIR, "bkf_present_wf.csv")

# ======================================================================
# SANITY + เตรียมคอลัมน์เป้าหมาย
# ======================================================================
if not isinstance(bkf_wfo, str):
    raise TypeError(f"bkf_wfo ควรเป็นชื่อคอลัมน์แบบ str แต่ได้ {type(bkf_wfo)}")

col_full = bkf_full if bkf_full else "wfo_full_name"
if col_full not in df_bkf_present.columns:
    df_bkf_present[col_full] = np.nan

def nonempty(s: pd.Series) -> pd.Series:
    return s.astype(str).str.strip().ne("")

# ======================================================================
# เตรียมรายการ WFO ที่จะดึง (เอา “ทั้งหมด” เพื่อให้เติมชื่อครบ 6571)
# ======================================================================
ids_all_unique = (df_bkf_present[bkf_wfo]
                  .dropna().astype(str).str.strip()
                  .drop_duplicates()
                  .tolist())
print(f"🔢 Unique WFO IDs in df_bkf_present: {len(ids_all_unique)}")

# ======================================================================
# โหลด/สร้าง CACHE และคำนวณคิวที่ยังต้องดึง
# ======================================================================
if os.path.exists(CACHE_CSV):
    cache_df = pd.read_csv(CACHE_CSV)
    if bkf_wfo not in cache_df.columns:
        for alt in ["wfo_id","wfo","wfoid","wfo_code"]:
            if alt in cache_df.columns:
                cache_df = cache_df.rename(columns={alt: bkf_wfo})
                break
else:
    cache_df = pd.DataFrame(columns=[bkf_wfo, "scraped_name", "scrape_status"])

done_ok = set(cache_df.loc[cache_df["scrape_status"]=="ok", bkf_wfo]) if not cache_df.empty else set()
ids_to_scrape = [i for i in ids_all_unique if i not in done_ok]
print(f"🧭 To do after cache check: {len(ids_to_scrape)}")

# ======================================================================
# ตั้งค่า Selenium (headless + polite UA + ปิดโหลดรูปให้ไวขึ้น)
# ======================================================================
chrome_opts = Options()
chrome_opts.add_argument("--headless=new")
chrome_opts.add_argument("--no-sandbox")
chrome_opts.add_argument("--disable-dev-shm-usage")
chrome_opts.add_argument("--disable-gpu")
chrome_opts.add_argument("--disable-blink-features=AutomationControlled")
chrome_opts.add_argument("--window-size=1280,900")
chrome_opts.add_argument("--lang=en-US")
chrome_opts.add_argument("--disable-features=NetworkService")
chrome_opts.add_argument("--user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X) "
                         "AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36")

# ปิดโหลดรูปเพื่อลดเวลาโหลดหน้า
chrome_opts.add_experimental_option("prefs", {
    "profile.managed_default_content_settings.images": 2
})

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_opts)
driver.set_page_load_timeout(WAIT_SEC + 5)

# ======================================================================
# Helpers
# ======================================================================
def atomic_write(df: pd.DataFrame, path: str):
    """เขียนไฟล์แบบ atomic กันไฟล์ค้างครึ่ง ถ้า interrupt"""
    tmp = f"{path}.tmp_{uuid.uuid4().hex}"
    df.to_csv(tmp, index=False)
    shutil.move(tmp, path)

def extract_name_from_html(html: str):
    soup = BeautifulSoup(html, "lxml")
    sci = soup.select_one("em[itemprop='scientificName'].taxonName")
    if not sci:
        sci = soup.select_one("em[itemprop='scientificName']") or soup.select_one("em.taxonName")
    auth = soup.select_one("span[itemprop='scientificNameAuthorship']")
    if not sci:
        return None
    sci_name  = sci.get_text(strip=True)
    auth_name = auth.get_text(strip=True) if auth else ""
    return f"{sci_name} {auth_name}".strip()

def polite_pause(extra=0.0):
    time.sleep(random.uniform(SLEEP_MIN, SLEEP_MAX) + extra)

def scrape_one_wfo(wfo_id: str):
    url = BASE_URL + str(wfo_id).strip()
    attempt = 0
    last_err = "unknown_error"
    while attempt < MAX_ATTEMPTS:
        attempt += 1
        try:
            driver.get(url)
            WebDriverWait(driver, WAIT_SEC).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, "em[itemprop='scientificName'], em.taxonName"))
            )
            time.sleep(0.4)  # เผื่อหน้าโหลดช้า (ลดลงเพื่อความเร็ว)
            full_name = extract_name_from_html(driver.page_source)
            if full_name:
                return full_name, "ok"
            raise TimeoutException("no_scientificName_tag")
        except (TimeoutException, WebDriverException) as e:
            last_err = f"retry_{attempt}: {e}"
            backoff = min(2.0 * attempt, 8.0) + random.uniform(0.5, 1.2)
            if any(k in str(e) for k in ["ERR_CONNEC", "ERR_TIMED_OUT", "ERR_FAILED",
                                          "ERR_CONNECTION_RESET","ERR_CONNECTION_CLOSED"]):
                backoff += 1.5
            polite_pause(extra=backoff)
        except Exception as e:
            return None, f"error: {e}"
    return None, last_err

# ======================================================================
# RUN SCRAPE
# ======================================================================
progress_rows = []
try:
    total = len(ids_to_scrape)
    print(f"🚀 Start scraping {total} WFO pages (polite mode)")
    for i, wfo_id in enumerate(ids_to_scrape, start=1):
        wfo_id = str(wfo_id).strip()
        if not wfo_id or wfo_id.lower() == "nan":
            progress_rows.append({bkf_wfo: wfo_id, "scraped_name": "", "scrape_status": "empty_wfo_id"})
            continue

        full_name, status = scrape_one_wfo(wfo_id)
        progress_rows.append({bkf_wfo: wfo_id, "scraped_name": full_name or "", "scrape_status": status})

        # หน่วงแบบสุภาพทุกครั้ง
        polite_pause()

        # Save cache/progress ทุก 10 แถว (หรือแถวสุดท้าย)
        if (i % SAVE_EVERY == 0) or (i == total):
            new_df = pd.DataFrame(progress_rows)

            # รวมเข้า cache และให้สถานะ ok ชนะกรณีซ้ำ
            cache_df = pd.concat([cache_df, new_df], ignore_index=True)
            cache_df.sort_values(by=["scrape_status"], key=lambda s: s.ne("ok"), inplace=True)
            cache_df = cache_df.drop_duplicates(subset=[bkf_wfo], keep="first")

            # เขียนไฟล์แบบ atomic
            atomic_write(cache_df, CACHE_CSV)
            atomic_write(new_df, PROGRESS_CSV)

            # flush stdout ให้เห็น log ทันที
            sys.stdout.flush()
            print(f"💾 Progress saved ({i}/{total}) -> {PROGRESS_CSV}")

            progress_rows = []
finally:
    try:
        driver.quit()
    except Exception:
        pass

# ======================================================================
# MERGE กลับเข้า df_bkf_present แล้วเติมชื่อให้ครบทุกแถว
# ======================================================================
ok_map = cache_df.loc[cache_df["scrape_status"]=="ok", [bkf_wfo, "scraped_name"]].rename(
    columns={"scraped_name": "__wfo_full_name_scraped__"}
)

# merge แบบ left เพื่อได้คอลัมน์ชื่อ scraped ชั่วคราว
df_bkf_present = df_bkf_present.merge(ok_map, on=bkf_wfo, how="left")

# เติมชื่อเต็มเฉพาะที่ยังว่าง/ว่างเปล่า
fill_mask = df_bkf_present[col_full].isna() | (~nonempty(df_bkf_present[col_full]))
df_bkf_present.loc[fill_mask, col_full] = df_bkf_present.loc[fill_mask, "__wfo_full_name_scraped__"]

# ลบคอลัมน์ช่วย
df_bkf_present.drop(columns=["__wfo_full_name_scraped__"], inplace=True, errors="ignore")

# ======================================================================
# SAVE ออกไฟล์ในโฟลเดอร์ Processed data
# ======================================================================
atomic_write(df_bkf_present, OUT_CSV)
print(f"✅ DONE. Saved df_bkf_present (with '{col_full}') -> {OUT_CSV}")

# รายงานจำนวนแถวที่มีชื่อเต็ม
rows_with_full = nonempty(df_bkf_present[col_full]).sum()
print(f"✨ Rows with non-empty '{col_full}': {rows_with_full} / {len(df_bkf_present)}")
# ======================================================================

🔢 Unique WFO IDs in df_bkf_present: 6571
🧭 To do after cache check: 15
🚀 Start scraping 15 WFO pages (polite mode)
💾 Progress saved (10/15) -> /Users/thanwaratkeratipasuwat/Desktop/dsi314/Processed data/bkf_present_wf_progress.csv
💾 Progress saved (15/15) -> /Users/thanwaratkeratipasuwat/Desktop/dsi314/Processed data/bkf_present_wf_progress.csv
✅ DONE. Saved df_bkf_present (with 'accepted_name') -> /Users/thanwaratkeratipasuwat/Desktop/dsi314/Processed data/bkf_present_wf.csv
✨ Rows with non-empty 'accepted_name': 6571 / 6571


## Mapping Name From bkf_present_cache.csv to bkf_present_wfo.csv

In [ ]:
# -*- coding: utf-8 -*-
import pandas as pd
import numpy as np
import os

# ======================
# 📂 PATH CONFIG
# ======================
BASE_DIR = "/Users/thanwaratkeratipasuwat/Desktop/dsi314/Processed data"
INPUT_WFO  = os.path.join(BASE_DIR, "bkf_present_wfo.csv")
INPUT_CACHE = os.path.join(BASE_DIR, "bkf_present_wf_cache.csv")
OUTPUT_FILE = os.path.join(BASE_DIR, "bkf_present_wfo_with_fullname.csv")

# ======================
# 🧩 LOAD FILES
# ======================
df_main = pd.read_csv(INPUT_WFO)
df_cache = pd.read_csv(INPUT_CACHE)

# ======================
# 🔍 เตรียมข้อมูล cache
# ======================
# เอาเฉพาะแถวที่ scrape สำเร็จ
df_ok = df_cache.loc[df_cache["scrape_status"] == "ok", ["wfo_id", "scraped_name"]].drop_duplicates("wfo_id")

# ======================
# 🧠 เติมชื่อเต็มใหม่
# ======================
df_main["wfo_full_name"] = df_main["wfo_id"].map(df_ok.set_index("wfo_id")["scraped_name"])

# ======================
# 💾 บันทึกไฟล์ใหม่
# ======================
df_main.to_csv(OUTPUT_FILE, index=False)
print(f"✅ Done! Saved new file with 'wfo_full_name' column → {OUTPUT_FILE}")

# (optional) ตรวจสอบสรุป
n_full = df_main["wfo_full_name"].notna().sum()
print(f"✨ Rows with full name: {n_full} / {len(df_main)}")

## BKF missing wfo_id 

In [ ]:
df_bkf_missing.shape

มี 82 แถวที่ไม่มีรายชื่อ เพราะอาจจะสุดที่ genus ค่ะ จาก 898

มี 157 ที่เป็น candidates อีก 650 คือมีชื่อแน่ๆ งั้นเราควรแทนชื่อแทนๆลงไปเลย 91 no_candidates found